<a href="https://colab.research.google.com/github/antonum/Redis-Workshops/blob/main/04-Large_Language_Model/04-Large_Language_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Large Language Models

![Redis](https://redis.com/wp-content/themes/wpx/assets/images/logo-redis.svg?auto=webp&quality=85,75&width=120)

In this notebook you'll be using two LLMs. OpenAI ChatGPT `gpt-3.5-turbo`and Self - hosted in - notebook `databricks/dolly-v2`. 

In [3]:
%pip -q install openai accelerate transformers sentence-transformers


Initialize OpenAI. You need to supply the OpenAI API key (starts with `sk-...`) when prompted.

In [4]:
import openai
import os
import getpass

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY","")
if OPENAI_API_KEY == "":
    key=getpass.getpass(prompt='OpenAI Key: ', stream=None)
    os.environ['OPENAI_API_KEY']=key
    
openai.api_key = os.getenv("OPENAI_API_KEY")

OpenAI Key: ··········


Initialize `databricks/dolly-v2-3b` via [HuggingFace](https://huggingface.co/databricks/dolly-v2-3b). Multiple progressively more powerful models are avaliable, including 3b, 7b and 12b (referring to Billions of parameters). `dolly-v2-3b` is the only model in the family that would fit in the memory and GPU available in a free Google Colab instance.

Loading and initializing the model can take few miutes.

In [5]:
import torch
from transformers import pipeline

generate_text = pipeline(model="databricks/dolly-v2-3b", 
                         torch_dtype=torch.bfloat16, 
                         trust_remote_code=True, 
                         device_map="auto")


Helper function for OpenAI ChatGPT model

In [11]:
def get_completion(prompt, model="gpt-3.5-turbo"):
    messages = [{"role": "user", "content": prompt}]
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        
        temperature=0, # this is the degree of randomness of the model's output
    )
    return response

# Create the prompt

Prompt contains instructions, context and the question. Feel free to exteriment with the prompt and see the difference in responses from different models.

In [15]:
context = """

Disney on Thursday upped the ante in its battle with Florida’s Republican Gov. Ron DeSantis, and it cost his state 2,000 white-collar jobs.

Disney is scrapping plans to build a $1 billion office complex in Florida, citing “changing business conditions,” according to a memo provided by a Disney spokesperson.

The decision comes at a time when the company is openly feuding with DeSantis, who is expected to officially enter the 2024 GOP presidential race next week, CNN reported Thursday.

A spokesperson for DeSantis said it was “unsurprising” that Disney would cancel the project “given the company’s financial straits, falling market cap and declining stock price.”

Disney, along with the broader media industry, is grappling with a difficult advertising environment and a massive writers strike. Earlier this year it announced it would be cutting 7,000 jobs as part of a cost-cutting effort.

Separately, the company confirmed Thursday that it would shut down its Star Wars: Galactic Starcruiser resort at Disney World just over a year after it opened.

The popular attraction “will take its final voyage” at the end of September, Disney said, adding that it is working with guests to rebook reservations for later in the year.
"""

question="What plans Disney is cancelling?"

prompt = f"""
Instruction: Use only information in the following context to answer the question at the end. 
If you don't know, say that you do not know. 
 
Context:  {context}
 
Question: {question}
 
Response:
"""
print(prompt)

res = generate_text(prompt)
print("Dolly:")
print(res[0]['generated_text'])


res = get_completion(prompt)
print("\nOpenAI:")
print(res.choices[0].message["content"])


Instruction: Use only information in the following context to answer the question at the end. 
If you don't know, say that you do not know. 
 
Context:  

Disney on Thursday upped the ante in its battle with Florida’s Republican Gov. Ron DeSantis, and it cost his state 2,000 white-collar jobs.

Disney is scrapping plans to build a $1 billion office complex in Florida, citing “changing business conditions,” according to a memo provided by a Disney spokesperson.

The decision comes at a time when the company is openly feuding with DeSantis, who is expected to officially enter the 2024 GOP presidential race next week, CNN reported Thursday.

A spokesperson for DeSantis said it was “unsurprising” that Disney would cancel the project “given the company’s financial straits, falling market cap and declining stock price.”

Disney, along with the broader media industry, is grappling with a difficult advertising environment and a massive writers strike. Earlier this year it announced it would b

## TODO:

Some ideas for you to try: 
- add "Respond in French/Spanish" to the prompt.

-  more information into the context until you hit the token limit of the model.

- Replace the entire prompt with a simple task like "Tell me about Newmarket, Ontario"

In [10]:
#generated_text("Tell me about Newmarket, Ontario")
#get_completion("Tell me about Newmarket, Ontario")